In [ ]:
from math import *
from numpy import *
from numpy.linalg import norm
import matplotlib.pyplot as plt
import ipywidgets as widgets
#%matplotlib inline
import scipy
import scipy.optimize
from scipy.optimize import fsolve
from scipy.misc import derivative
atm=1.01325E5
torr=atm/760.0
R=8.3145
g=9.807
NA=6.022e23
kB=R/NA
def tok(x): return x+273.15


In [ ]:
#Functions used

def safelog(x): return ma.log(x).filled(-1e9)  # The value of log(0) doesn't matter as it's multiplied by 0 anyway

def pop(levels, T):
    T+=1e-3  #Avoid numerical errors when T=0 by adding a negligible value
    q=sum(exp(-levels/T))
    p=exp(-levels/T)/q
    U=sum(p*levels)
    S=-sum(p*safelog(p))
    return (q,p,U,S)


def setEnergy(U,levels):  #Returns the temperature that corresponds to energy U
    return fsolve(lambda T: pop(levels,T)[2]-float(U), 1.0)

def energytransfer(deltaU, TA, TB, levelsA, levelsB):   #Move deltaU (which can be positive or negative) from system A to system B
    (_,_,UA,_)=pop(levelsA,TA)
    (_,_,UB,_)=pop(levelsB,TB)
    UA-=deltaU  
    UB+=deltaU
    if UA>=0 and UB>=0:
        TA=setEnergy(UA,levelsA)
        TB=setEnergy(UB,levelsB)
        return (True,TA,TB)
    else:
        return (False,TA,TB)

def equidistant(distance): 
    return arange(1000)*float(distance)

def sublevels(numsublevels, maindistance, subdistance):
    lev=[]
    for i in range(1000//numsublevels):
        for j in range(numsublevels):
            lev.append(i*float(maindistance)+j*float(subdistance))
    lev.sort()
    return array(lev)

In [ ]:
#System definitions

#Uncomment the desired variant (equidistant or sublevels) for system A and B, respectively

#levels_A=equidistant(10)  #Unit K
levels_A=sublevels(5,100,10)  #Unit K

levels_B=equidistant(100)
#levels_A=sublevels(5,100,10)  #Unit K

#Give either starting temperature or energy (in K units) for each system

T_A=75
#T_A=setEnergy(90.0,levels_A)

T_B=100
#T_B=setEnergy(90.0,levels_B)


In [ ]:
nshow_A=10
nshow_B=10
continuous=True   #Update graphs while sliding temperatures. Set to false if lagging is too severe.

def update(TA,TB, relativeA, relativeB):
    (q_A,p_A,U_A,S_A)=pop(levels_A,TA)
    (q_B,p_B,U_B,S_B)=pop(levels_B,TB)
    if relativeA:
        p_A/=p_A[0]
    if relativeB:
        p_B/=p_B[0]
    textA='S/k (particle): %.4f\nS (molar): %.4f J/(K mol)\nq: %.4f\nU/k (particle): %.4f K\nU (molar): %.4f kJ/mol\n'%(S_A,S_A*R,q_A,U_A,U_A*R*1e-3)
    textB='S/k (particle): %.4f\nS (molar): %.4f J/(K mol)\nq: %.4f\nU/k (particle): %.4f K\nU (molar): %.4f kJ/mol\n'%(S_B,S_B*R,q_B,U_B,U_B*R*1e-3)

    plt.figure(figsize=(16, 6))
    plt.clf()
    plt.subplot(1,2,1)
    plt.xlim([0,1])
    plt.barh(levels_A[:nshow_A],p_A[:nshow_A],height=0.5*(levels_A[1]-levels_A[0]))
    plt.annotate(textA, xy=(0.65, 0.7), xycoords='axes fraction')
    plt.title('System A')
    plt.xlabel('Population')
    plt.ylabel(r'Energy $E_i\; /\; k_B$ / (K$^{-1}$)')
    
    plt.subplot(1,2,2)
    plt.xlim([0,1])
    plt.barh(levels_B[:nshow_B],p_B[:nshow_B],height=0.5*(levels_B[1]-levels_B[0]))
    plt.annotate(textB, xy=(0.65, 0.7), xycoords='axes fraction')
    plt.title('System B')
    plt.xlabel('Population')
    plt.ylabel(r'Energy $E_i\; /\; k_B$ / (K$^{-1}$)')
    plt.show()

def transfer_left(b):
    (success,TA,TB)=energytransfer(-edit.value*1e3/R, slider1.value, slider2.value,levels_A, levels_B)
    if success:
        slider1.value=TA
        slider2.value=TB
        statuslabel.value=""
    else:
        statuslabel.value="IMPOSSIBLE!"

def transfer_right(b):
    (success,TA,TB)=energytransfer(edit.value*1e3/R, slider1.value, slider2.value,levels_A, levels_B)
    if success:
        slider1.value=TA
        slider2.value=TB
        statuslabel.value=""
    else:
        statuslabel.value="IMPOSSIBLE!"
        
slider1 = widgets.FloatSlider(description='T_A (K)',value=T_A,min=0,max=1000,step=1, continuous_update = continuous)
slider2 = widgets.FloatSlider(description='T_B (K)',value=T_B,min=0,max=1000,step=1, continuous_update = continuous)
box1 = widgets.Checkbox(False, description='relative first level (A)')
box2 = widgets.Checkbox(False, description='relative first level (B)')
button1=widgets.Button(description='<<')
edit=widgets.BoundedFloatText(value=0.1,min=0,max=10.0,step=0.01,description='Heat flow:',disabled=False, width=5)
statuslabel=widgets.Label(value="")
button2=widgets.Button(description='>>')
button1.on_click(transfer_left)
button2.on_click(transfer_right)
output=widgets.interactive_output(update, {'TA':slider1, 'TB': slider2, 'relativeA' : box1, 'relativeB' : box2})
display(widgets.HBox([slider1, slider2]))
display(widgets.HBox([button1, edit, widgets.Label(value="kJ/mol"),button2, statuslabel]))
display(widgets.HBox([box1, box2]))
display(output)
